# Curatarea, procesarea si crearea tabelului final

In [78]:
import pandas as pd
import numpy as np
import json
import os

In [79]:
school_details= pd.read_csv('school_details.csv')

In [101]:
levels_formations=pd.read_csv('school_levels_formations.csv')

In [81]:
tempo_all=pd.read_csv('Tempo_all.csv')

In [83]:
school_details.columns

Index(['idSchool', 'internalIdSchool', 'idParentSchool',
       'schoolYearDescription', 'code', 'siruesCode', 'longName', 'shortName',
       'schoolType', 'statut', 'isPj', 'fiscalCode', 'operatingMode',
       'propertyForm', 'fundingForm', 'county', 'locality', 'street',
       'streetNumber', 'postalCode', 'phoneNumber', 'faxNumber', 'email',
       'idSchoolYear.code', 'schoolNumbers.idSchool',
       'schoolNumbers.studyFormationsCount', 'schoolNumbers.studentsCount',
       'schoolNumbers.personnelCount'],
      dtype='object')

In [84]:
df=school_details[['idSchool', 'county', 'schoolNumbers.studentsCount', 'schoolNumbers.personnelCount']]

In [85]:
school_details= df.rename(columns = {'idSchool': 'idSchool', 'county': 'Judet', 'schoolNumbers.studentsCount': 'Nr_elevi',
                                'schoolNumbers.personnelCount':'Nr_profesori'})

In [86]:
len(school_details[school_details.Judet.isna()])

0

In [87]:
len(school_details[school_details.Nr_elevi.isna()])

0

In [88]:
len(school_details[school_details.Nr_profesori.isna()])

0

In [89]:
# verificam daca exista valori dublicate pe idSchool
len(school_details[school_details.idSchool.duplicated()])

0

In [91]:
# creeam un tabel de mapare diacriticelor
diacr = {"Ă":"A", "Ş":"S","Ţ":"T","Â":"A","Î":"I"}

translation_table = str.maketrans(diacr)

# inlocuim diacriticele din coloana cu judete
school_details['Judet'] = school_details['Judet'].str.translate(translation_table)

In [92]:
school_details.Judet.unique()

array(['ARAD', 'BUZAU', 'CALARASI', 'MEHEDINTI', 'TULCEA', 'IALOMITA',
       'SIBIU', 'CARAS-SEVERIN', 'BISTRITA-NASAUD', 'ALBA', 'HARGHITA',
       'ARGES', 'BOTOSANI', 'BRASOV', 'BRAILA', 'MUNICIPIUL BUCURESTI',
       'CLUJ', 'CONSTANTA', 'DOLJ', 'GALATI', 'SUCEAVA', 'GIURGIU',
       'BACAU', 'HUNEDOARA', 'ILFOV', 'MARAMURES', 'NEAMT', 'MURES',
       'BIHOR', 'PRAHOVA', 'SATU MARE', 'SALAJ', 'COVASNA', 'VRANCEA',
       'OLT', 'IASI', 'DAMBOVITA', 'TELEORMAN', 'GORJ', 'TIMIS', 'VALCEA',
       'VASLUI'], dtype=object)

In [103]:
levels_formations.head()

,id,idSchool,sfName,sfLevel,studyFormationType,educationType,fundingForm,educationForm,teachingMode,teachingLanguage,techingType,studentsNo
0,930621,11269910,B Îngerașii,Preșcolar,Grupa mică,Masă,Sponsorizare,Normal,Tradițional,Limba română,Normal,0
1,930620,11269910,A Ursuletii,Preșcolar,Grupa mică,Masă,Sponsorizare,Normal,Tradițional,Limba română,Normal,0
2,930623,11269910,A,Preșcolar,Grupa mare,Masă,Sponsorizare,Normal,Tradițional,Limba română,Normal,0
3,930622,11269910,A,Preșcolar,Grupa mijlocie,Masă,Sponsorizare,Normal,Tradițional,Limba română,Normal,0
4,828033,11263948,A,Gimnazial,Clasa a VI-a,Special,Buget,Zi,Pedagogia curativă,Limba română,Normal,0


In [95]:
# verificam dac aexista NA-uri
len(levels_formations[levels_formations.idSchool.isna()])

0

In [100]:
# verificam daca exista dublicate
len(levels_formations[levels_formations.idSchool.duplicated()])

149157

In [104]:
levels_formations.drop(columns= ['id', 'sfName', 'studyFormationType', 'educationType', 'fundingForm', 
                                 'educationForm', 'teachingMode', 'techingType', 'studentsNo'], inplace=True)

In [106]:
levels_formations.head()

,idSchool,sfLevel,teachingLanguage
0,11269910,Preșcolar,Limba română
1,11269910,Preșcolar,Limba română
2,11269910,Preșcolar,Limba română
3,11269910,Preșcolar,Limba română
4,11263948,Gimnazial,Limba română


In [107]:
levels_formations.groupby('sfLevel')['idSchool'].nunique()

sfLevel
Antepreşcolar       23
Gimnazial         6354
Liceal            1505
Postliceal         590
Preșcolar        11051
Primar            8805
Profesional        800
Name: idSchool, dtype: int64

In [108]:
levels_formations= levels_formations[levels_formations['sfLevel']=='Liceal']

In [109]:
levels_formations.sfLevel.unique()

array(['Liceal'], dtype=object)

In [110]:
len(levels_formations)

24701

In [111]:
# verificam daca exista dublicate pe toate coloanele
len(levels_formations[levels_formations.duplicated()])

23119

In [112]:
# eliminam dublicatele
levels_formations= levels_formations.drop_duplicates()
len(levels_formations[levels_formations.duplicated()])

0

In [113]:
# verificam daca exista dublicate pe idSchool
len(levels_formations[levels_formations.idSchool.duplicated()])

77

In [114]:
len(levels_formations[levels_formations[['idSchool', 'sfLevel']].duplicated()])

77

In [115]:
# analizam scolile pentru care se dubleaza idSchool:
levels_formations[levels_formations.idSchool.duplicated()].head(4)

,idSchool,sfLevel,teachingLanguage
2084,11257839,Liceal,Limba polonă
2490,11266873,Liceal,Limba maghiară
3127,11260674,Liceal,Limba maghiară
4500,11258242,Liceal,Limba germană


In [116]:
# dublicatele sunt pe idSchool sunt la scolile care au mai mult de o limba de predare:
levels_formations[levels_formations['idSchool']==11257839]

,idSchool,sfLevel,teachingLanguage
2064,11257839,Liceal,Limba română
2084,11257839,Liceal,Limba polonă


#### pentru a elimina dublicatele transformam coloana teachingLanguage in mai multe coloane cu valori dummy, exemplu:
##### coloana 'romana': daca scoala are ca limba de predare romana=1, daca nu=0
##### coloana 'engleza': daca are predare in engleza=1, daca nu=0

### Transformam coloana teachingLanguage in mai multe coloane

In [117]:
levels_formations.teachingLanguage.unique()

array(['Limba română', 'Limba maghiară', 'Limba polonă', 'Limba germană',
       'Limba ucraineană', 'Limba croată', 'Limba neogreacă',
       'Limba bulgară', 'Limba rusă', 'Limba romani (rromani)',
       'Limba italiană', 'Limba sârbă', 'Limba engleză', 'Limba slovacă'],
      dtype=object)

In [118]:
# creeam un tabel de mapare diacriticelor
diacr = {"ă":"a", "ș":"s","ț":"t","â":"a","î":"i"}

translation_table = str.maketrans(diacr)

# inlocuim diacriticele din coloana cu limbile
levels_formations['teachingLanguage'] = levels_formations['teachingLanguage'].str.translate(translation_table)

In [119]:
levels_formations.teachingLanguage.unique()

array(['Limba romana', 'Limba maghiara', 'Limba polona', 'Limba germana',
       'Limba ucraineana', 'Limba croata', 'Limba neogreaca',
       'Limba bulgara', 'Limba rusa', 'Limba romani (rromani)',
       'Limba italiana', 'Limba sarba', 'Limba engleza', 'Limba slovaca'],
      dtype=object)

In [120]:
dummy_columns=pd.get_dummies(levels_formations['teachingLanguage']).astype(int)

In [121]:
dummy_columns.head()

,Limba bulgara,Limba croata,Limba engleza,Limba germana,Limba italiana,Limba maghiara,Limba neogreaca,Limba polona,Limba romana,Limba romani (rromani),Limba rusa,Limba sarba,Limba slovaca,Limba ucraineana
46,0,0,0,0,0,0,0,0,1,0,0,0,0,0
64,0,0,0,0,0,0,0,0,1,0,0,0,0,0
131,0,0,0,0,0,0,0,0,1,0,0,0,0,0
266,0,0,0,0,0,0,0,0,1,0,0,0,0,0
390,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [122]:
levels_formations= pd.concat([levels_formations, dummy_columns], axis=1)

In [124]:
levels_formations.head()

,idSchool,sfLevel,teachingLanguage,Limba bulgara,Limba croata,Limba engleza,Limba germana,Limba italiana,Limba maghiara,Limba neogreaca,Limba polona,Limba romana,Limba romani (rromani),Limba rusa,Limba sarba,Limba slovaca,Limba ucraineana
46,11274703,Liceal,Limba romana,0,0,0,0,0,0,0,0,1,0,0,0,0,0
64,11257683,Liceal,Limba romana,0,0,0,0,0,0,0,0,1,0,0,0,0,0
131,11274296,Liceal,Limba romana,0,0,0,0,0,0,0,0,1,0,0,0,0,0
266,11274297,Liceal,Limba romana,0,0,0,0,0,0,0,0,1,0,0,0,0,0
390,11274299,Liceal,Limba romana,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [125]:
# analizam tranfsormarea pentru un liceu care are 2 limbi de predare
levels_formations[levels_formations['idSchool']==11257839]

,idSchool,sfLevel,teachingLanguage,Limba bulgara,Limba croata,Limba engleza,Limba germana,Limba italiana,Limba maghiara,Limba neogreaca,Limba polona,Limba romana,Limba romani (rromani),Limba rusa,Limba sarba,Limba slovaca,Limba ucraineana
2064,11257839,Liceal,Limba romana,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2084,11257839,Liceal,Limba polona,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [126]:
levels_formations.drop(columns=['sfLevel', 'teachingLanguage'], inplace=True)

In [41]:
levels_formations[levels_formations['idSchool']==11257839]

,idSchool,Limba bulgara,Limba croata,Limba engleza,Limba germana,Limba italiana,Limba maghiara,Limba neogreaca,Limba polona,Limba romana,Limba romani (rromani),Limba rusa,Limba sarba,Limba slovaca,Limba ucraineana
2064,11257839,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2084,11257839,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [127]:
licee= levels_formations.groupby('idSchool').sum().reset_index()
licee.head(3)

,idSchool,Limba bulgara,Limba croata,Limba engleza,Limba germana,Limba italiana,Limba maghiara,Limba neogreaca,Limba polona,Limba romana,Limba romani (rromani),Limba rusa,Limba sarba,Limba slovaca,Limba ucraineana
0,11255240,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,11255594,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,11255631,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [128]:
licee[licee['idSchool']==11257839]

,idSchool,Limba bulgara,Limba croata,Limba engleza,Limba germana,Limba italiana,Limba maghiara,Limba neogreaca,Limba polona,Limba romana,Limba romani (rromani),Limba rusa,Limba sarba,Limba slovaca,Limba ucraineana
91,11257839,0,0,0,0,0,0,0,1,1,0,0,0,0,0


In [129]:
len(licee)

1505

In [130]:
len(licee[licee.duplicated()])

0

In [131]:
licee.columns

Index(['idSchool', 'Limba bulgara', 'Limba croata', 'Limba engleza',
       'Limba germana', 'Limba italiana', 'Limba maghiara', 'Limba neogreaca',
       'Limba polona', 'Limba romana', 'Limba romani (rromani)', 'Limba rusa',
       'Limba sarba', 'Limba slovaca', 'Limba ucraineana'],
      dtype='object')

In [132]:
licee.rename(columns= {'idSchool':'idSchool', 'Limba bulgara': 'bulgara', 'Limba croata' : 'croata',
                       'Limba engleza': 'engleza','Limba germana':'germana', 'Limba italiana':'italiana',
                       'Limba maghiara': 'maghiara' , 'Limba neogreaca':'neogreaca', 
                       'Limba polona': 'polona', 'Limba romana': 'romana', 
                       'Limba romani (rromani)':'romani', 'Limba rusa': 'rusa','Limba sarba':'sarba',
                       'Limba slovaca':'slovaca', 'Limba ucraineana': 'ucraineana'}, inplace=True)

In [133]:
licee.head()

,idSchool,bulgara,croata,engleza,germana,italiana,maghiara,neogreaca,polona,romana,romani,rusa,sarba,slovaca,ucraineana
0,11255240,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,11255594,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,11255631,0,0,0,0,0,0,0,0,1,0,0,0,0,0
3,11255635,0,0,0,0,0,1,0,0,1,0,0,0,0,0
4,11255657,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [134]:
school_details.head()

,idSchool,Judet,Nr_elevi,Nr_profesori
0,11269910,ARAD,131,7
1,11256876,BUZAU,0,0
2,11256865,BUZAU,0,0
3,11284187,CALARASI,0,0
4,11270113,CALARASI,0,0


In [135]:
# unim tabelele licee si school details)
all_details= pd.merge(licee,school_details, on= 'idSchool', how='left')

In [136]:
all_details.head()

,idSchool,bulgara,croata,engleza,germana,italiana,maghiara,neogreaca,polona,romana,romani,rusa,sarba,slovaca,ucraineana,Judet,Nr_elevi,Nr_profesori
0,11255240,0,0,0,0,0,0,0,0,1,0,0,0,0,0,VALCEA,959,94
1,11255594,0,0,0,0,0,0,0,0,1,0,0,0,0,0,COVASNA,361,42
2,11255631,0,0,0,0,0,0,0,0,1,0,0,0,0,0,COVASNA,1072,110
3,11255635,0,0,0,0,0,1,0,0,1,0,0,0,0,0,COVASNA,994,118
4,11255657,0,0,0,0,0,1,0,0,0,0,0,0,0,0,COVASNA,505,63


In [137]:
len(all_details)

1505

In [138]:
all_details.columns

Index(['idSchool', 'bulgara', 'croata', 'engleza', 'germana', 'italiana',
       'maghiara', 'neogreaca', 'polona', 'romana', 'romani', 'rusa', 'sarba',
       'slovaca', 'ucraineana', 'Judet', 'Nr_elevi', 'Nr_profesori'],
      dtype='object')

In [139]:
aggregations={'idSchool': 'count',
              'bulgara' : 'sum',
              'croata': 'sum', 
              'engleza': 'sum', 
              'germana': 'sum', 
              'italiana': 'sum',
              'maghiara': 'sum', 
              'neogreaca': 'sum', 
              'polona': 'sum', 
              'romana': 'sum',
              'romani': 'sum', 
              'rusa': 'sum', 
              'sarba': 'sum',
              'slovaca': 'sum', 
              'ucraineana': 'sum',
              'Nr_elevi': 'sum', 
              'Nr_profesori': 'sum'}
all_details=all_details.groupby(['Judet']).agg(aggregations).reset_index()

In [141]:
all_details

,Judet,idSchool,bulgara,croata,engleza,germana,italiana,maghiara,neogreaca,polona,romana,romani,rusa,sarba,slovaca,ucraineana,Nr_elevi,Nr_profesori
0,ALBA,36,0,0,0,1,0,2,0,0,33,0,0,0,0,0,21419,2416
1,ARAD,41,0,0,0,1,0,1,0,0,40,0,0,0,1,0,31643,3166
2,ARGES,43,0,0,0,0,0,0,0,0,43,0,0,0,0,0,29802,3057
3,BACAU,36,0,0,0,0,0,0,0,0,36,0,0,0,0,0,27915,3055
4,BIHOR,51,0,0,0,1,0,14,0,0,42,0,0,0,1,0,41575,4430
5,BISTRITA-NASAUD,27,0,0,0,1,0,1,0,0,27,0,0,0,0,0,19115,1973
6,BOTOSANI,29,0,0,0,0,0,0,0,0,29,0,0,0,0,0,21813,2387
7,BRAILA,22,0,0,0,0,0,0,0,0,22,0,0,0,0,0,15657,1623
8,BRASOV,42,0,0,0,3,0,4,0,0,39,0,0,0,0,0,31486,3270
9,BUZAU,33,0,0,0,0,0,0,0,0,33,0,0,0,0,0,20968,2319


In [142]:
tempo_all.head()

,Judet,Elevi_cu_BAC,Ateliere_scolare,Laboratoare,Echipamente,Terenuri_sport,Imigranti
0,Bihor,2915,108,253,6137,73,300
1,Bistrita-Nasaud,1553,47,104,3814,21,177
2,Cluj,3653,112,368,14700,75,692
3,Maramures,2449,67,135,6815,37,326
4,Satu Mare,1460,74,139,3722,39,205


In [143]:
all_details.Judet.unique()

array(['ALBA', 'ARAD', 'ARGES', 'BACAU', 'BIHOR', 'BISTRITA-NASAUD',
       'BOTOSANI', 'BRAILA', 'BRASOV', 'BUZAU', 'CALARASI',
       'CARAS-SEVERIN', 'CLUJ', 'CONSTANTA', 'COVASNA', 'DAMBOVITA',
       'DOLJ', 'GALATI', 'GIURGIU', 'GORJ', 'HARGHITA', 'HUNEDOARA',
       'IALOMITA', 'IASI', 'ILFOV', 'MARAMURES', 'MEHEDINTI',
       'MUNICIPIUL BUCURESTI', 'MURES', 'NEAMT', 'OLT', 'PRAHOVA',
       'SALAJ', 'SATU MARE', 'SIBIU', 'SUCEAVA', 'TELEORMAN', 'TIMIS',
       'TULCEA', 'VALCEA', 'VASLUI', 'VRANCEA'], dtype=object)

In [144]:
tempo_all.Judet.unique()

array([' Bihor', ' Bistrita-Nasaud', ' Cluj', ' Maramures', ' Satu Mare',
       ' Salaj', ' Alba', ' Brasov', ' Covasna', ' Harghita', ' Mures',
       ' Sibiu', ' Bacau', ' Botosani', ' Iasi', ' Neamt', ' Suceava',
       ' Vaslui', ' Braila', ' Buzau', ' Constanta', ' Galati', ' Tulcea',
       ' Vrancea', ' Arges', ' Calarasi', ' Dambovita', ' Giurgiu',
       ' Ialomita', ' Prahova', ' Teleorman', ' Ilfov',
       ' Municipiul Bucuresti', ' Dolj', ' Gorj', ' Mehedinti', ' Olt',
       ' Valcea', ' Arad', ' Caras-Severin', ' Hunedoara', ' Timis'],
      dtype=object)

In [145]:
# elemininam spatiile
tempo_all['Judet']= tempo_all['Judet'].str.strip()

In [146]:
tempo_all['Judet']= tempo_all['Judet'].str.upper()

In [147]:
tempo_all.Judet.unique()

array(['BIHOR', 'BISTRITA-NASAUD', 'CLUJ', 'MARAMURES', 'SATU MARE',
       'SALAJ', 'ALBA', 'BRASOV', 'COVASNA', 'HARGHITA', 'MURES', 'SIBIU',
       'BACAU', 'BOTOSANI', 'IASI', 'NEAMT', 'SUCEAVA', 'VASLUI',
       'BRAILA', 'BUZAU', 'CONSTANTA', 'GALATI', 'TULCEA', 'VRANCEA',
       'ARGES', 'CALARASI', 'DAMBOVITA', 'GIURGIU', 'IALOMITA', 'PRAHOVA',
       'TELEORMAN', 'ILFOV', 'MUNICIPIUL BUCURESTI', 'DOLJ', 'GORJ',
       'MEHEDINTI', 'OLT', 'VALCEA', 'ARAD', 'CARAS-SEVERIN', 'HUNEDOARA',
       'TIMIS'], dtype=object)

In [148]:
licee_all=pd.merge(all_details, tempo_all, on='Judet', how='inner')

In [149]:
licee_all

,Judet,idSchool,bulgara,croata,engleza,germana,italiana,maghiara,neogreaca,polona,...,slovaca,ucraineana,Nr_elevi,Nr_profesori,Elevi_cu_BAC,Ateliere_scolare,Laboratoare,Echipamente,Terenuri_sport,Imigranti
0,ALBA,36,0,0,0,1,0,2,0,0,...,0,0,21419,2416,1817,82,162,4882,37,215
1,ARAD,41,0,0,0,1,0,1,0,0,...,1,0,31643,3166,2047,86,209,3759,51,315
2,ARGES,43,0,0,0,0,0,0,0,0,...,0,0,29802,3057,3416,129,241,5320,40,162
3,BACAU,36,0,0,0,0,0,0,0,0,...,0,0,27915,3055,3165,100,206,6239,43,1250
4,BIHOR,51,0,0,0,1,0,14,0,0,...,1,0,41575,4430,2915,108,253,6137,73,300
5,BISTRITA-NASAUD,27,0,0,0,1,0,1,0,0,...,0,0,19115,1973,1553,47,104,3814,21,177
6,BOTOSANI,29,0,0,0,0,0,0,0,0,...,0,0,21813,2387,2182,61,121,3415,30,4609
7,BRAILA,22,0,0,0,0,0,0,0,0,...,0,0,15657,1623,1388,81,104,2568,19,105
8,BRASOV,42,0,0,0,3,0,4,0,0,...,0,0,31486,3270,2668,97,203,4005,40,634
9,BUZAU,33,0,0,0,0,0,0,0,0,...,0,0,20968,2319,2046,37,140,3178,20,129


In [150]:
licee_all.idSchool.sum()

1505

In [65]:
licee_all.to_csv('licee_all.csv', index=False)

## Incarcarea in Azure a tabelului final

In [69]:
import pyodbc
import sqlalchemy as db
import urllib

exec(open('azure_credentials.py').read())

In [66]:
database_name = "AID2023"
driver = '{ODBC Driver 18 for SQL Server}'
server_name = "aid-2023-srv.database.windows.net"

In [70]:
# Create the server url
server = 'tcp:{server_name},1433'.format(server_name = server_name)

#create the full connection string <- make sure to write it as it is bellow - check your credentials to be the right ones

connection_string = 'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};charset=utf8'.format(
    driver = driver,
    server=server,
    database=database_name,
    username=username,
    password=password
)                              

cnxn: pyodbc.Connection = pyodbc.connect(connection_string)
cursor: pyodbc.Cursor = cnxn.cursor() 

#execute a query with the cursor to check if the connection is maade
print("Connection is open") if cursor.execute("SELECT 1") else print(f"Connection is closed: {e}")

Connection is open


In [71]:
#obtain the parameters from the pyodbc connection string
params = urllib.parse.quote_plus(connection_string)
#create the engine
sqlalchemy_engine = db.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [72]:
# inserarea datelor intr-un tabel sql
licee_all.to_sql("licee_all_team4", sqlalchemy_engine, if_exists='replace', index=False)  

42